# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("C:\\Users\\arond\\Desktop\\MSU Bootcamp\\Class_Work\\Module6\\Challenge\\python-api-challenge\\WeatherPy\\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hotan,37.0997,79.9269,64.54,41,100,18.05,CN,1695257172
1,1,ust-nera,64.5667,143.2000,31.66,79,100,9.95,RU,1695257172
2,2,iqaluit,63.7506,-68.5145,29.93,100,100,2.30,CA,1695257046
3,3,le mee-sur-seine,48.5316,2.6283,68.18,78,75,14.97,FR,1695257172
4,4,longyearbyen,78.2186,15.6401,28.24,80,20,2.30,SJ,1695257095


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_data = city_data_df.hvplot.points("Lat","Lng", size="Humidity", color="City", scale=1)

# Display the map
map_data

:Points   [Lat,Lng]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
temperature_criteria = (city_data_df["Max Temp"] >= 70) & (city_data_df["Max Temp"] <= 80)
humidity_criteria = city_data_df["Humidity"] <= 50
cloudiness_criteria = city_data_df["Cloudiness"] <= 20
wind_speed_criteria = city_data_df["Wind Speed"] <= 10

filtered_cities_df = city_data_df[temperature_criteria & humidity_criteria & cloudiness_criteria & wind_speed_criteria]


# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
sample_data = filtered_cities_df.head()
sample_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
177,177,zagora,30.3484,-5.8365,77.40,17,0,2.55,MA,1695257306
192,192,carinhanha,-14.3047,-43.7650,79.81,33,0,7.43,BR,1695257337
258,258,francistown,-21.1700,27.5079,70.84,33,0,7.49,BW,1695257425
290,290,sydney,-33.8679,151.2073,75.54,30,0,5.99,AU,1695257069
487,487,al jawf,29.5000,38.7500,71.55,47,0,8.66,SA,1695257670


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
sample_data = hotel_df.head()
sample_data

,City,Country,Lat,Lng,Humidity,Hotel Name
177,zagora,MA,30.3484,-5.8365,17,
192,carinhanha,BR,-14.3047,-43.7650,33,
258,francistown,BW,-21.1700,27.5079,33,
290,sydney,AU,-33.8679,151.2073,30,
487,al jawf,SA,29.5000,38.7500,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "type": "hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lat},{lng}"
    params["bias"] = f"circle:{radius}@{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
zagora - nearest hotel: No hotel found
carinhanha - nearest hotel: No hotel found
francistown - nearest hotel: No hotel found
sydney - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
hami - nearest hotel: No hotel found
hwange - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
177,zagora,MA,30.3484,-5.8365,17,No hotel found
192,carinhanha,BR,-14.3047,-43.7650,33,No hotel found
258,francistown,BW,-21.1700,27.5079,33,No hotel found
290,sydney,AU,-33.8679,151.2073,30,No hotel found
487,al jawf,SA,29.5000,38.7500,47,No hotel found
511,hami,CN,42.8000,93.4500,15,No hotel found
567,hwange,ZW,-18.3693,26.5019,34,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
city_map_with_hotels = hotel_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    colorbar=True,
    title='City Locations with Hotel Info and Humidity',
    xlabel='Longitude',
    ylabel='Latitude',
    hover_cols=["City", "Humidity", "Hotel Name", "Country"],  # Include "Hotel Name" and "Country" in the hover message
    size='Humidity',
    width=800,
    height=500
)

# Display the map
city_map_with_hotels

:Scatter   [Lng]   (Lat,Humidity,City,Hotel Name,Country)